<a href="https://colab.research.google.com/github/jsansao/dlvc/blob/main/Licao20_IdentificacaoFacial_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lição 20 - Identificação facial com fine tuning do VGGFace2




## Download do dataset

In [1]:
!wget https://github.com/jsansao/fame-dataset/archive/refs/heads/main.zip

--2021-11-27 17:58:22--  https://github.com/jsansao/fame-dataset/archive/refs/heads/main.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/jsansao/fame-dataset/zip/refs/heads/main [following]
--2021-11-27 17:58:22--  https://codeload.github.com/jsansao/fame-dataset/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.112.10
Connecting to codeload.github.com (codeload.github.com)|140.82.112.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘main.zip’

main.zip                [        <=>         ]  47.24M  29.7MB/s    in 1.6s    

2021-11-27 17:58:24 (29.7 MB/s) - ‘main.zip’ saved [49530242]



In [2]:
!unzip main.zip

Archive:  main.zip
e76125873a9351f49ba1f84baf8e2ff90f3731ab
   creating: fame-dataset-main/
   creating: fame-dataset-main/train/
   creating: fame-dataset-main/train/AlPacino/
  inflating: fame-dataset-main/train/AlPacino/1.al_pacino.jpg  
  inflating: fame-dataset-main/train/AlPacino/11.29373_0d16849303cb824.jpg  
  inflating: fame-dataset-main/train/AlPacino/12.210px-al_pacino_roma_film_fest_cropped.jpg  
  inflating: fame-dataset-main/train/AlPacino/13.image-w856.jpg  
  inflating: fame-dataset-main/train/AlPacino/14.os-80-anos-de-al-pacino0298839100202004251215.jpg  
  inflating: fame-dataset-main/train/AlPacino/15.filmes-com-al-pacino-pdb-117%e2%80%8b.jpg  
  inflating: fame-dataset-main/train/AlPacino/19.al-pacino.jpg  
  inflating: fame-dataset-main/train/AlPacino/2.1337912.jpg  
  inflating: fame-dataset-main/train/AlPacino/23.al_pacino_2016_%2830401544240%29.jpg  
  inflating: fame-dataset-main/train/AlPacino/25.al_pacino.jpg  
 extracting: fame-dataset-main/train/AlPacino/26

## Extração das faces do dataset, geração dos conjuntos de treinamento e validação




In [3]:
!pip install mtcnn

     |████████████████████████████████| 2.3 MB 5.4 MB/s 


In [4]:
# face detection for the 5 Celebrity Faces Dataset
from os import listdir
from os.path import isdir
from PIL import Image
from matplotlib import pyplot
from numpy import savez_compressed
from numpy import asarray
import mtcnn

# extract a single face from a given photograph
def extract_face(filename, required_size=(224, 224)):
	# load image from file
	image = Image.open(filename)
	# convert to RGB, if needed
	image = image.convert('RGB')
	# convert to array
	pixels = asarray(image)
	# create the detector, using default weights
	detector = mtcnn.MTCNN()
	# detect faces in the image
	results = detector.detect_faces(pixels)
	# extract the bounding box from the first face
	x1, y1, width, height = results[0]['box']
	# bug fix
	x1, y1 = abs(x1), abs(y1)
	x2, y2 = x1 + width, y1 + height
	# extract the face
	face = pixels[y1:y2, x1:x2]
	# resize pixels to the model size
	image = Image.fromarray(face)
	image = image.resize(required_size)
	face_array = asarray(image)
	return face_array

# load images and extract faces for all images in a directory
def load_faces(directory):
	faces = list()
	# enumerate files
	for filename in listdir(directory):
		# path
		path = directory + filename
		# get face
		face = extract_face(path, (224,224))
		# store
		faces.append(face)
	return faces

# load a dataset that contains one subdir for each class that in turn contains images
def load_dataset(directory):
	X, y = list(), list()
	# enumerate folders, on per class
	for subdir in listdir(directory):
		# path
		path = directory + subdir + '/'
		# skip any files that might be in the dir
		if not isdir(path):
			continue
		# load all faces in the subdirectory
		faces = load_faces(path)
		# create labels
		labels = [subdir for _ in range(len(faces))]
		# summarize progress
		print('>loaded %d examples for class: %s' % (len(faces), subdir))
		# store
		X.extend(faces)
		y.extend(labels)
	return asarray(X), asarray(y)



In [35]:
# load train dataset
trainX, trainy = load_dataset('/content/fame-dataset-main/train/')
print(trainX.shape, trainy.shape)
# load test dataset
testX, testy = load_dataset('/content/fame-dataset-main/val/')
# save arrays to one file in compressed format
savez_compressed('fame-faces-dataset.npz', trainX, trainy, testX, testy)

>loaded 15 examples for class: AlPacino
>loaded 15 examples for class: EmilyBlunt
>loaded 15 examples for class: AngelinaJolie
>loaded 15 examples for class: KristenStewart
>loaded 20 examples for class: NicolasCage
>loaded 15 examples for class: MerylStreep
>loaded 15 examples for class: ClintEastwood
>loaded 15 examples for class: TomHanks
(125, 224, 224, 3) (125,)
>loaded 2 examples for class: AlPacino
>loaded 3 examples for class: EmilyBlunt
>loaded 3 examples for class: AngelinaJolie
>loaded 11 examples for class: KristenStewart
>loaded 9 examples for class: NicolasCage
>loaded 11 examples for class: MerylStreep
>loaded 11 examples for class: ClintEastwood
>loaded 4 examples for class: TomHanks


## Codificação dos rótulos

In [42]:
from sklearn.preprocessing import LabelEncoder
out_encoder = LabelEncoder()
out_encoder.fit(trainy)
trainY = out_encoder.transform(trainy)
testY = out_encoder.transform(testy)




## Instação do VGGFACE

In [7]:


!pip install git+https://github.com/jsansao/keras-vggface.git
!pip install keras_applications



  Cloning https://github.com/jsansao/keras-vggface.git to /tmp/pip-req-build-9ab16nst
  Running command git clone -q https://github.com/jsansao/keras-vggface.git /tmp/pip-req-build-9ab16nst
  Created wheel for keras-vggface: filename=keras_vggface-0.6-py3-none-any.whl size=8315 sha256=b4241b88acc22c4f37737e8df600bf53481ec043e63028a7d5d6ada78611393a
  Stored in directory: /tmp/pip-ephem-wheel-cache-jy_i3w25/wheels/17/1d/de/3357313e72504b373dcd31226714c9477da009000aeb307586
Successfully built keras-vggface
     |████████████████████████████████| 50 kB 2.9 MB/s 


In [34]:


from numpy import expand_dims
from matplotlib import pyplot
from PIL import Image
from numpy import asarray

import mtcnn
from keras.utils.layer_utils import get_source_inputs

from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from keras_vggface.utils import decode_predictions

from keras.layers import Flatten, Dense, Input



## Carregamento do modelo pré-treinado, sem camadas de saída

In [19]:
nb_class = 8
hidden_dim = 512



model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
model.trainable = False
last_layer = model.get_layer('avg_pool').output
x = Flatten(name='flatten')(last_layer)
out = Dense(nb_class, activation='softmax', name='classifier')(x)
custom_vgg_model = tensorflow.keras.Model(model.input, out)

custom_vgg_model.summary()

Model: "model_540"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_545 (InputLayer)         [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1/7x7_s2 (Conv2D)          (None, 112, 112, 64  9408        ['input_545[0][0]']              
                                )                                                                 
                                                                                                  
 conv1/7x7_s2/bn (BatchNormaliz  (None, 112, 112, 64  256        ['conv1/7x7_s2[0][0]']           
 ation)                         )                                                         

## Treinamento das camadas de saída

In [41]:

from tensorflow.keras.optimizers import Adam

optimizer = Adam(learning_rate=0.001)
custom_vgg_model.compile(loss="sparse_categorical_crossentropy", metrics=['accuracy'], optimizer=optimizer)


In [44]:
history = custom_vgg_model.fit( trainX, trainY, validation_data=(testX, testY),   epochs=20)

Epoch 1/20
4/4 [==============================] - 1s 320ms/step - loss: 6.2019e-06 - accuracy: 1.0000 - val_loss: 0.1384 - val_accuracy: 0.9815
Epoch 2/20
4/4 [==============================] - 1s 300ms/step - loss: 5.5948e-06 - accuracy: 1.0000 - val_loss: 0.1392 - val_accuracy: 0.9815
Epoch 3/20
4/4 [==============================] - 1s 295ms/step - loss: 5.3107e-06 - accuracy: 1.0000 - val_loss: 0.1396 - val_accuracy: 0.9815
Epoch 4/20
4/4 [==============================] - 1s 296ms/step - loss: 5.0972e-06 - accuracy: 1.0000 - val_loss: 0.1400 - val_accuracy: 0.9815
Epoch 5/20
4/4 [==============================] - 1s 299ms/step - loss: 4.9180e-06 - accuracy: 1.0000 - val_loss: 0.1402 - val_accuracy: 0.9815
Epoch 6/20
4/4 [==============================] - 1s 300ms/step - loss: 4.7054e-06 - accuracy: 1.0000 - val_loss: 0.1403 - val_accuracy: 0.9815
Epoch 7/20
4/4 [==============================] - 1s 297ms/step - loss: 4.6043e-06 - accuracy: 1.0000 - val_loss: 0.1404 - val_accuracy:

## Fine-tuning 

In [46]:
model.trainable = True

custom_vgg_model.summary()

Model: "model_540"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_545 (InputLayer)         [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1/7x7_s2 (Conv2D)          (None, 112, 112, 64  9408        ['input_545[0][0]']              
                                )                                                                 
                                                                                                  
 conv1/7x7_s2/bn (BatchNormaliz  (None, 112, 112, 64  256        ['conv1/7x7_s2[0][0]']           
 ation)                         )                                                         

In [47]:
optimizer = Adam(learning_rate=1e-5)
custom_vgg_model.compile(loss="sparse_categorical_crossentropy", metrics=['accuracy'], optimizer=optimizer)


In [48]:
history = custom_vgg_model.fit( trainX, trainY, validation_data=(testX, testY),   epochs=20)

Epoch 1/20
4/4 [==============================] - 12s 1s/step - loss: 0.0181 - accuracy: 0.9920 - val_loss: 0.1456 - val_accuracy: 0.9815
Epoch 2/20
4/4 [==============================] - 3s 716ms/step - loss: 3.8503e-04 - accuracy: 1.0000 - val_loss: 0.1530 - val_accuracy: 0.9815
Epoch 3/20
4/4 [==============================] - 3s 712ms/step - loss: 3.3721e-04 - accuracy: 1.0000 - val_loss: 0.1563 - val_accuracy: 0.9815
Epoch 4/20
4/4 [==============================] - 3s 714ms/step - loss: 2.9454e-04 - accuracy: 1.0000 - val_loss: 0.1567 - val_accuracy: 0.9815
Epoch 5/20
4/4 [==============================] - 3s 715ms/step - loss: 1.9300e-04 - accuracy: 1.0000 - val_loss: 0.1539 - val_accuracy: 0.9815
Epoch 6/20
4/4 [==============================] - 3s 717ms/step - loss: 4.1891e-05 - accuracy: 1.0000 - val_loss: 0.1483 - val_accuracy: 0.9815
Epoch 7/20
4/4 [==============================] - 3s 715ms/step - loss: 8.0384e-05 - accuracy: 1.0000 - val_loss: 0.1430 - val_accuracy: 0.981

## Teste do modelo

In [49]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


Saving 28.actor-nicolas-cage-attends-the-special-screening-of-color-out-of-at-picture-id1199567621.jpg to 28.actor-nicolas-cage-attends-the-special-screening-of-color-out-of-at-picture-id1199567621.jpg
User uploaded file "28.actor-nicolas-cage-attends-the-special-screening-of-color-out-of-at-picture-id1199567621.jpg" with length 36627 bytes


In [ ]:
# load the photo and extract the face
pixels = extract_face(fn)
# plot the extracted face
pyplot.imshow(pixels)
# show the plot
pyplot.show()
# convert one face into samples
pixels = pixels.astype('float32')
samples = expand_dims(pixels, axis=0)
# prepare the face for the model, e.g. center pixels
samples = preprocess_input(samples, version=2)

# perform prediction
yhat = custom_vgg_model.predict(samples)
